In [1]:
import pandas as pd
import warnings

#warnings.filterwarnings("ignore")

from epilepsy_prediction import load_prediction_model, data_preprocess,fuse_string,evaluate_model,load_imputation_model
from epilepsy_prediction.imputation import column_imputer
from copy import deepcopy
from pandas import read_csv
from pandas import DataFrame
from pandas import melt
from pandas import merge
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression

def load_data(change_name_file,data_file_name,outcome_col="outcome"):
    data_df = pd.read_csv(data_file_name, index_col=0)
    new_name_df = pd.read_csv(change_name_file,index_col=0)
    data_df.rename(new_name_df['marketscan'].to_dict(),inplace=True,axis=1)
    y_train = data_df.loc[data_df["test_train"]=="train",outcome_col]
    X_train = data_df.loc[data_df["test_train"]=="train",~( data_df.columns.isin([outcome_col,"test_train"]))]
    y_test = data_df.loc[~(data_df["test_train"]=="train"),outcome_col]
    X_test = data_df.loc[~(data_df["test_train"]=="train"),~ (data_df.columns.isin([outcome_col,"test_train"]))]
    return X_train,y_train,X_test,y_test


## all drugs, outcome any

In [2]:
change_name_file=r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\dictionary_19.01.23.csv"
data_file_name = r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\Data\test_train_drugs_all.csv"
X_train,y_train,X_test,y_test = load_data(change_name_file,data_file_name, outcome_col="outcome_any")

cols=X_train.columns[-6:]
X = X_train.drop("outcome_primary", axis = 1)  
Xs = X_test.drop("outcome_primary", axis = 1)  
X.sort_index(inplace=True)
Xs.sort_index(inplace=True)

stbl = X_train[cols].sum().reset_index().set_axis(['drug', 'count'], axis=1).assign(weight=lambda df: df['count'])

wtbl = merge(
    melt(X_train[cols].reset_index(), id_vars=['id'], var_name='drug')
    .pipe(lambda df: df[df['value'] == 1])
    [['id', 'drug']],
    stbl,
    on='drug',
    how='left'
)[['id', 'weight']].set_index('id')

weights = wtbl.loc[X.index.values, 'weight']

##XGBoost
xgbc = XGBClassifier()
model_xg = xgbc.fit(X, y_train.loc[X.index], sample_weight=weights)

##Logistic
##lr = LogisticRegression()
##model_lr = lr.fit(X, y_train.loc[X.index], sample_weight=weights)

In [3]:
#AUC test
probs_xg = model_xg.predict_proba(Xs)[:, 1]
auc_xg = roc_auc_score(y_test.loc[Xs.index], probs_xg)
auc_xg

0.5421315404074025

In [4]:
#AUC train
probs_xg = model_xg.predict_proba(X)[:, 1]
auc_xg = roc_auc_score(y_train.loc[X.index], probs_xg)
auc_xg

1.0

## all drugs, outcome primary

In [5]:
change_name_file=r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\dictionary_19.01.23.csv"
data_file_name = r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\Data\test_train_drugs_all.csv"
X_train,y_train,X_test,y_test = load_data(change_name_file,data_file_name, outcome_col="outcome_primary")

cols=X_train.columns[-6:]
X = X_train.drop("outcome_any", axis = 1)  
Xs = X_test.drop("outcome_any", axis = 1)  
X.sort_index(inplace=True)
Xs.sort_index(inplace=True)

stbl = X_train[cols].sum().reset_index().set_axis(['drug', 'count'], axis=1).assign(weight=lambda df: df['count'])

wtbl = merge(
    melt(X_train[cols].reset_index(), id_vars=['id'], var_name='drug')
    .pipe(lambda df: df[df['value'] == 1])
    [['id', 'drug']],
    stbl,
    on='drug',
    how='left'
)[['id', 'weight']].set_index('id')

weights = wtbl.loc[X.index.values, 'weight']

##XGBoost
xgbc = XGBClassifier()
model_xg = xgbc.fit(X, y_train.loc[X.index], sample_weight=weights)

##Logistic
##lr = LogisticRegression()
##model_lr = lr.fit(X, y_train.loc[X.index], sample_weight=weights)

In [6]:
#AUC test
probs_xg = model_xg.predict_proba(Xs)[:, 1]
auc_xg = roc_auc_score(y_test.loc[Xs.index], probs_xg)
auc_xg

0.5830064402810304

In [7]:
#AUC train
probs_xg = model_xg.predict_proba(X)[:, 1]
auc_xg = roc_auc_score(y_train.loc[X.index], probs_xg)
auc_xg

1.0

## 1st index any

In [8]:
change_name_file=r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\dictionary_19.01.23.csv"
data_file_name = r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\Data\test_train_drugs_1st_index.csv"
X_train,y_train,X_test,y_test = load_data(change_name_file,data_file_name, outcome_col="outcome_any")

cols=X_train.columns[-6:]
X = X_train.drop("outcome_primary", axis = 1)  
Xs = X_test.drop("outcome_primary", axis = 1)  
X.sort_index(inplace=True)
Xs.sort_index(inplace=True)

stbl = X_train[cols].sum().reset_index().set_axis(['drug', 'count'], axis=1).assign(weight=lambda df: df['count'])

wtbl = merge(
    melt(X_train[cols].reset_index(), id_vars=['id'], var_name='drug')
    .pipe(lambda df: df[df['value'] == 1])
    [['id', 'drug']],
    stbl,
    on='drug',
    how='left'
)[['id', 'weight']].set_index('id')

weights = wtbl.loc[X.index.values, 'weight']

##XGBoost
xgbc = XGBClassifier()
model_xg = xgbc.fit(X, y_train.loc[X.index], sample_weight=weights)

##Logistic
##lr = LogisticRegression()
##model_lr = lr.fit(X, y_train.loc[X.index], sample_weight=weights)

In [9]:
#AUC test
probs_xg = model_xg.predict_proba(Xs)[:, 1]
auc_xg = roc_auc_score(y_test.loc[Xs.index], probs_xg)
auc_xg

0.5206204718558425

In [10]:
#AUC train
probs_xg = model_xg.predict_proba(X)[:, 1]
auc_xg = roc_auc_score(y_train.loc[X.index], probs_xg)
auc_xg

1.0

## 1st primary

In [11]:
change_name_file=r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\dictionary_19.01.23.csv"
data_file_name = r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\Data\test_train_drugs_1st_index.csv"
X_train,y_train,X_test,y_test = load_data(change_name_file,data_file_name, outcome_col="outcome_primary")

cols=X_train.columns[-6:]
X = X_train.drop("outcome_any", axis = 1)  
Xs = X_test.drop("outcome_any", axis = 1)  
X.sort_index(inplace=True)
Xs.sort_index(inplace=True)

stbl = X_train[cols].sum().reset_index().set_axis(['drug', 'count'], axis=1).assign(weight=lambda df: df['count'])

wtbl = merge(
    melt(X_train[cols].reset_index(), id_vars=['id'], var_name='drug')
    .pipe(lambda df: df[df['value'] == 1])
    [['id', 'drug']],
    stbl,
    on='drug',
    how='left'
)[['id', 'weight']].set_index('id')

weights = wtbl.loc[X.index.values, 'weight']

##XGBoost
xgbc = XGBClassifier()
model_xg = xgbc.fit(X, y_train.loc[X.index], sample_weight=weights)

##Logistic
##lr = LogisticRegression()
##model_lr = lr.fit(X, y_train.loc[X.index], sample_weight=weights)

In [12]:
#AUC test
probs_xg = model_xg.predict_proba(Xs)[:, 1]
auc_xg = roc_auc_score(y_test.loc[Xs.index], probs_xg)
auc_xg

0.5849262317899229

In [13]:
#AUC train
probs_xg = model_xg.predict_proba(X)[:, 1]
auc_xg = roc_auc_score(y_train.loc[X.index], probs_xg)
auc_xg

1.0